<a href="https://colab.research.google.com/github/Sidor-Sprengmeister/Hac-novosib/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Untitled0_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# lex rank - unsupervised upproach
!pip install sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.nlp.stemmers import Stemmer
import nltk
from nltk.corpus import stopwords
import numpy as np


# nltk.download('stopwords')

# Допольнительные стоп-слова можно скачать здесь
# https://github.com/stopwords-iso/stopwords-ru/blob/master/raw/stop-words-russian.txt
# но в этот список мы также добавили пару примеров вручную, поэтому прикладываем готовый файл.
# Вы также можете модифицировать на свое усмотрение, или вовсе от него отказаться

with open("stop-words-russian.txt", 'r') as f:
    extra_stop_words = f.readlines()
    extra_stop_words = [line.strip() for line in extra_stop_words]


def sumy_method(text, n_sent: int = 4):

    parser = PlaintextParser.from_string(text, Tokenizer("russian"))

    stemmer = Stemmer("russian")
    summarizer = LexRankSummarizer(stemmer)
    stopwords_ru = stopwords.words('russian')
    stopwords_ru.extend(extra_stop_words)
    summarizer.stop_words = stopwords_ru

    #Summarize the document with n_sent sentences
    summary = summarizer(parser.document, n_sent)
    dp = []
    if len(summary)> 0:
        for i in summary:
            lp = str(i)
            dp.append(lp)

        final_sentence = ' '.join(dp)
    else:
        final_sentence = ''
    if len(final_sentence.split(" "))>512:
        final_sentence = " ".join(final_sentence.split(" ")[:512])
    return final_sentence

In [ ]:
import pandas as pd
import os
PATH_TO_DATA = 'train/'
dataset = pd.read_csv(os.path.join(PATH_TO_DATA, "train.csv"))

In [ ]:
dataset.head(5)

,video_name,stt_name,category_name,title,description
0,0.mp4,0.txt,Развлечения,Правильная цена I #3,С вами Макс Климток и это шоу Правильная цена!...
1,1.mp4,1.txt,Спорт/Игры,Три лошадиные силы | Выпуск №2,В этом новом выпуске нас ждут не менее новые и...
2,2.mp4,2.txt,Блоги,Хашлама | Выпуск 4 | Силиконовый ПРЕСС Давы | ...,"Привет, это Султан и Авет! Мы опять хаваем вку..."
3,3.mp4,3.txt,Путешествия,Прогулка по стране - Владивосток,Прогулка по Владивостоку. Самому большому горо...
4,4.mp4,4.txt,Искусство,Артмеханика. Выпуск 3. Татуировки + Mika Vino,Были ли татуировки на теле Николая II? Почему ...


In [ ]:
dataset[dataset.video_name == '478.mp4']

,video_name,stt_name,category_name,title,description
478,478.mp4,478.txt,Путешествия,Прогулка по стране - Екатеринбург,Прогулка по Екатеринбургу — третьему по величи...


In [ ]:
with open(os.path.join(PATH_TO_DATA, 'train_stt', '478.txt'), 'r') as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines]
lines

['[75.18s -> 1590.92s]  С вами был Игорь Негода.']

In [ ]:
import nltk
from nltk.translate import meteor
from nltk import word_tokenize, sent_tokenize
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import nltk
nltk.download('punkt')
dataset['len'] = dataset.description.apply(lambda l : len(sent_tokenize(l)))
print("Среднее число предложений в трейн датасете", np.mean(dataset['len'].to_list()))
print("Медиана", np.median(dataset['len'].to_list()))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Среднее число предложений в трейн датасете 4.032
Медиана 3.0


In [ ]:
# Очистим STT от временных кодов
from tqdm import tqdm
tqdm.pandas()
def del_timestamps(text):
    text = text.split("]  ")[1:]
    return " ".join(text)

In [ ]:
def gen_description(stt_name, n_sent, category_name):

    with open(os.path.join(PATH_TO_DATA, 'train_stt', stt_name), 'r') as f:
        lines = f.readlines()
        lines = [del_timestamps(line.strip()) for line in lines]
        lines = " ".join(lines)
        res = sumy_method(lines, n_sent)
        if len(res)>0:
            return res
        else:
            return category_name

In [ ]:
%%time
dataset['stt_sum'] = np.nan
dataset['stt_sum'] = dataset.progress_apply(lambda l: gen_description(l.stt_name, 4, l.category_name), axis=1)

100%|██████████| 500/500 [14:39<00:00,  1.76s/it]

CPU times: user 14min 31s, sys: 37.6 s, total: 15min 9s
Wall time: 14min 39s


In [ ]:
dataset[dataset.stt_sum.isin(dataset.category_name.unique())]

,video_name,stt_name,category_name,title,description,len,stt_sum,meteor,meteor_first4
31,31.mp4,31.txt,Авто-мото,DSC OFF на Байкальской миле 2020,"ГАЗ-24 «Волга КГБ» — проект Гурама Инцкирвели,...",4,Авто-мото,0.0,0.0
74,74.mp4,74.txt,Путешествия,Прогулка по стране - Казань,"Сегодня нам предстоит прогулка по городу, кото...",3,Путешествия,0.0,0.0
111,111.mp4,111.txt,Спорт,Команда MOTORCITY на Байкальской миле 2020,MOTORCITY собрала на фестиваль скорости “Байка...,5,Спорт,0.0,0.0
178,178.mp4,178.txt,Путешествия,Прогулка по стране - Москва - часть 1,Первый выпуск проекта “Прогулки по стране” пос...,1,Путешествия,0.0,0.0
212,212.mp4,212.txt,Спорт,Команда FCM Racing Team на Байкальской миле 2020,Эта команда привезла на Байкал соль Бонневилля...,5,Спорт,0.0,0.0
298,298.mp4,298.txt,Авто-мото,IVECO грузовик на Байкальской миле 2020,"Фестиваль скорости ""Байкальская Миля 2020"" соб...",4,Авто-мото,0.0,0.0


In [ ]:
dataset.head(3)

,video_name,stt_name,category_name,title,description,len,stt_sum,meteor,meteor_first4
0,0.mp4,0.txt,Развлечения,Правильная цена I #3,С вами Макс Климток и это шоу Правильная цена!...,3,"Итак, после первого раунда третье место с резу...",0.1008,0.0389
1,1.mp4,1.txt,Спорт/Игры,Три лошадиные силы | Выпуск №2,В этом новом выпуске нас ждут не менее новые и...,4,"Так. Короче, чтобы выходом надо сделать задани...",0.0742,0.0340
2,2.mp4,2.txt,Блоги,Хашлама | Выпуск 4 | Силиконовый ПРЕСС Давы | ...,"Привет, это Султан и Авет! Мы опять хаваем вку...",5,"Короче, Авет, сколько всего произошло на прошл...",0.0954,0.0448


In [ ]:
dataset.stt_sum.to_list()[:3]

['Итак, после первого раунда третье место с результатом два балла у нас занимает Кика. Картина стоит в долларах, не в рублях. Давай, делаем вторую картину. Не переживай, меня это не волнует ни разу, так что давай делать.',
 'Так. Короче, чтобы выходом надо сделать задание. Но с этими заданиями это вообще жопа. Что с Аветом, я не понимаю, он грузит видос уже где-то минут 25 и не может загрузить. Короче, давай одна минута.',
 'Короче, Авет, сколько всего произошло на прошлой неделе ты даже не в курсе. Потому что у тебя на лице и так черные точки. Ее надо типа убивать, да, вообще? А зачем мы это смотрим?']

In [ ]:
# dataset.to_csv("train_with_generated_sum.csv")

In [ ]:
def func(stt_name, text, text_sum):
    if isinstance(text_sum, str):
        return round(meteor([word_tokenize(text)],word_tokenize(text_sum)), 4)
    else:
        return 0
dataset['meteor'] = dataset.apply(lambda l: func(l['stt_name'], l.description, l.stt_sum), axis=1)

In [ ]:
print("Значение метрики meteor для unsupervised модели", dataset.meteor.mean())
# метрика в данной реализации имеет значения от 0 до 1

Значение метрики meteor для unsupervised модели 0.0997602


In [ ]:
%%time
def func(stt_name, text, category_name):
    with open(os.path.join(PATH_TO_DATA, 'train_stt', stt_name), 'r') as f:
        lines = f.readlines()
        lines = [del_timestamps(line.strip()) for line in lines]
        res = lines[:4]
    res = " ".join(lines)
    if isinstance(res, str):
        return round(meteor([word_tokenize(text)],word_tokenize(res)), 4)
    else:
        return round(meteor([word_tokenize(text)],word_tokenize(category_name)), 4)
dataset['meteor_first4'] = dataset.apply(lambda l: func(l['stt_name'], l.description, l.category_name), axis=1)

CPU times: user 1min 33s, sys: 255 ms, total: 1min 33s
Wall time: 1min 34s


In [ ]:
print("Значение метрики meteor для модели, выдающей первые 4 предложения", dataset.meteor_first4.mean())

Значение метрики meteor для модели, выдающей первые 4 предложения 0.057719


In [ ]:
import pandas as pd
import os
PATH_TO_DATA = 'test/'
dataset = pd.read_csv(os.path.join(PATH_TO_DATA, "test.csv"))

In [ ]:
dataset.head(5)

,video_name,stt_name,category_name
0,0.mp4,0.txt,Новости и СМИ
1,1.mp4,1.txt,Развлечения
2,2.mp4,2.txt,Развлечения
3,3.mp4,3.txt,Видеоигры
4,4.mp4,4.txt,Развлечения


In [ ]:
dataset[dataset.video_name == '2.mp4']

,video_name,stt_name,category_name
2,2.mp4,2.txt,Развлечения


In [ ]:
with open(os.path.join(PATH_TO_DATA, 'test_stt', '2.txt'), 'r') as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines]
lines

In [ ]:
import nltk
from nltk.translate import meteor
from nltk import word_tokenize, sent_tokenize
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True